In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture(0) # 0 -Default Camera/Webcam

while True: #to keep the video capture running continously
    _,frame = cap.read()
    frame = cv2.flip(frame,1) # 1 - Horizontal flip
    
    cv2.imshow("frame",frame)
    
    key = cv2.waitKey(1)
    
    if key == 27: # 27 - ESC key
        break
        
cap.release()
cv2.destroyAllWindows()

In [3]:
#creating trackbars to find the desired hsv range for our objects
#play with the sliders a bit to create a threshold for the objects

import numpy as np
import cv2

def nothing(x):
    pass

cap = cv2.VideoCapture(0)

cv2.namedWindow("Trackbars")

cv2.createTrackbar('LowerHue', 'Trackbars', 0, 179, nothing)
cv2.createTrackbar('LowerSat', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('LowerVal', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('UpperHue', 'Trackbars', 179, 179, nothing)
cv2.createTrackbar('UpperSat', 'Trackbars', 255, 255, nothing)
cv2.createTrackbar('UpperVal', 'Trackbars', 255, 255, nothing)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    blurredframe = cv2.GaussianBlur(frame,(5,5),0)
    
    hsv = cv2.cvtColor(blurredframe, cv2.COLOR_BGR2HSV)

    l_h = cv2.getTrackbarPos('LowerHue', 'Trackbars')
    l_s = cv2.getTrackbarPos('LowerSat', 'Trackbars')
    l_v = cv2.getTrackbarPos('LowerVal', 'Trackbars')
    u_h = cv2.getTrackbarPos('UpperHue', 'Trackbars')
    u_s = cv2.getTrackbarPos('UpperSat', 'Trackbars')
    u_v = cv2.getTrackbarPos('UpperVal', 'Trackbars')

    lower = np.array([l_h, l_s, l_v])
    upper = np.array([u_h, u_s, u_v])
    mask = cv2.inRange(hsv, lower, upper)

    result = cv2.bitwise_and(frame, frame, mask=mask)

    cv2.imshow('frame',frame)
    cv2.imshow('mask', mask)
    cv2.imshow('result', result)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
#range for the colors
lower_orange = np.array([20, 47, 204])
upper_orange = np.array([36, 232, 255])

lower_pink = np.array([165,72,69])
upper_pink = np.array([179,98,255])

In [ ]:
import cv2
import datetime
import numpy as np
import time
from pynput.mouse import Button, Controller

#resizing so that our frame and the browser can be visible on screen at the same time
def resize(image, width =None, height =None, inter =cv2.INTER_AREA):
    dim = None
    (h,w) = image.shape[:2]
    
    
    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)

    return resized


cap = cv2.VideoCapture(0)


m = Controller()

last_click = datetime.datetime.now()

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    
    lower_orange = np.array([20, 47, 204])
    upper_orange = np.array([36, 232, 255])
    mask_orange = cv2.inRange(hsv, lower_orange, upper_orange)
    
    
    lower_pink = np.array([153,33,255])
    upper_pink = np.array([179,164,255])
    mask_pink = cv2.inRange(hsv, lower_pink, upper_pink)
    


    contoursOrange, _ = cv2.findContours(mask_orange, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contoursOrange:
        if cv2.contourArea(c) >= 50:
            if len(c) > 0:
                cnt = sorted(contoursOrange, key = cv2.contourArea, reverse = True)[0]
                rect = np.int32(cv2.boxPoints(cv2.minAreaRect(c)))
                cv2.drawContours(frame, [rect], -1, (0, 255, 0), 2)
                x, y, _, _ = cv2.boundingRect(cnt)
                m.position=(x,y)
       
        
    contoursPink, _ = cv2.findContours(mask_pink, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contoursPink :
        if cv2.contourArea(c) >= 50 :
            now = datetime.datetime.now()
            diff = now - last_click
            if diff.total_seconds() > 0.5 :
                last_click = datetime.datetime.now()
                if len(c) > 0:
                    cv2.drawContours(frame, contoursPink, -1, (0, 255, 0), 3)
                    m.press(Button.left)
                    m.release(Button.left)
        
        
    frame = resize(frame, width = 400)  
    cv2.imshow("frame", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()